In [11]:
!pip install google-cloud-vision pillow pandas python-docx scikit-learn -q

In [12]:
import os
import math
from collections import Counter
from google.cloud import vision
import re

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='credentials.json'
client = vision.ImageAnnotatorClient()
WORD = re.compile(r"\w+")

In [5]:
import os

# Chemin vers le dossier
dossier = "piccolomini - good crop"

# Parcourir tous les fichiers du dossier
for filename in os.listdir(dossier):
    # Vérifier si le nom du fichier contient au moins 3 chiffres à la fin
    if filename[-7:-4].isdigit():
        # Extraire les trois derniers chiffres
        nouveau_nom = filename[-7:-4] + filename[-4:]

        # Construire les chemins complets pour l'ancien et le nouveau nom
        ancien_chemin = os.path.join(dossier, filename)
        nouveau_chemin = os.path.join(dossier, nouveau_nom)

        # Renommer le fichier
        os.rename(ancien_chemin, nouveau_chemin)
        print(f"Renommé: {filename} -> {nouveau_nom}")
    else:
        print(f"Ignoré: {filename} (ne correspond pas au format attendu)")

print("Renommage terminé.")

Renommé: Piccolomini_page_16.jpg -> 016.jpg
Renommé: Piccolomini_page_26.jpg -> 026.jpg
Renommé: Piccolomini_page_4.jpg -> 004.jpg
Renommé: Piccolomini_page_45.jpg -> 045.jpg
Renommé: Piccolomini_page_13.jpg -> 013.jpg
Renommé: Piccolomini_page_29.jpg -> 029.jpg
Ignoré: .ipynb_checkpoints (ne correspond pas au format attendu)
Renommé: Piccolomini_page_6.jpg -> 006.jpg
Renommé: Piccolomini_page_23.jpg -> 023.jpg
Renommé: Piccolomini_page_30.jpg -> 030.jpg
Renommé: Piccolomini_page_18.jpg -> 018.jpg
Renommé: Piccolomini_page_46.jpg -> 046.jpg
Renommé: Piccolomini_page_5.jpg -> 005.jpg
Renommé: Piccolomini_page_19.jpg -> 019.jpg
Renommé: Piccolomini_page_14.jpg -> 014.jpg
Renommé: Piccolomini_page_38.jpg -> 038.jpg
Renommé: Piccolomini_page_36.jpg -> 036.jpg
Renommé: Piccolomini_page_10.jpg -> 010.jpg
Renommé: Piccolomini_page_11.jpg -> 011.jpg
Renommé: Piccolomini_page_7.jpg -> 007.jpg
Renommé: Piccolomini_page_42.jpg -> 042.jpg
Renommé: Piccolomini_page_9.jpg -> 009.jpg
Renommé: Piccolo

In [19]:
import io
import os
from google.cloud import vision
from docx import Document
from docx.shared import Inches
from docx.enum.section import WD_ORIENT
from PIL import Image

def detect_text(path):
    client = vision.ImageAnnotatorClient()
    with open(path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.document_text_detection(image=image)
    texts = response.text_annotations
    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return texts[0].description if texts else ""

def process_images(folder_path):
    doc = Document()

    # Change orientation to landscape
    section = doc.sections[0]
    section.orientation = WD_ORIENT.LANDSCAPE
    section.page_width, section.page_height = section.page_height, section.page_width

    # Find min_i and max_i
    min_i = 1000
    max_i = -1
    for i in range(1000):
        image_name = f"{i:03d}.jpg"
        image_path = os.path.join(folder_path, image_name)
        if os.path.exists(image_path):
            min_i = min(min_i, i)
            max_i = max(max_i, i)

    if min_i > max_i:
        print("No images found in the specified folder.")
        return

    table = doc.add_table(rows=1, cols=2)
    table.style = 'Table Grid'
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = f'Pages from {min_i:03d} to {max_i:03d}'
    hdr_cells[1].text = 'OCR Text'

    for i in range(min_i, max_i + 1):
        image_name = f"{i:03d}.jpg"
        image_path = os.path.join(folder_path, image_name)

        if os.path.exists(image_path):
            ocr_text = detect_text(image_path)

            row_cells = table.add_row().cells

            # Insert image
            paragraph = row_cells[0].paragraphs[0]
            run = paragraph.add_run()
            run.add_picture(image_path, width=Inches(3))  # Adjust width as needed

            row_cells[1].text = ocr_text

    doc.save('output.docx')
    print(f"OCR results and images from {min_i:03d} to {max_i:03d} have been saved to output.docx in landscape orientation")

# Main execution
folder_path = "/content/folder"
process_images(folder_path)

OCR results and images from 017 to 018 have been saved to output.docx in landscape orientation


In [18]:
import io
import os
from google.cloud import vision
from docx import Document
from docx.shared import Inches
from docx.enum.section import WD_ORIENT
from PIL import Image

def detect_text(path):
    client = vision.ImageAnnotatorClient()
    with open(path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.document_text_detection(image=image)
    texts = response.text_annotations
    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return texts[0].description if texts else ""

def process_images(folder_path):
    doc = Document()

    # Landscape orientation
    section = doc.sections[0]
    section.orientation = WD_ORIENT.LANDSCAPE
    section.page_width, section.page_height = section.page_height, section.page_width

    table = doc.add_table(rows=1, cols=2)
    table.style = 'Table Grid'
    hdr_cells = table.rows[0].cells

    for i in range(1000):  # Loop from 000 to 999
        image_name = f"{i:03d}.jpg"
        image_path = os.path.join(folder_path, image_name)

        if os.path.exists(image_path):
            ocr_text = detect_text(image_path)

            row_cells = table.add_row().cells

            # Insert image
            paragraph = row_cells[0].paragraphs[0]
            run = paragraph.add_run()
            run.add_picture(image_path, width=Inches(4))

            row_cells[1].text = ocr_text

    doc.save('output.docx')
    print("OCR results and images have been saved to output.docx in landscape orientation")

# Main execution
folder_path = "/content/folder"
process_images(folder_path)

OCR results and images have been saved to output.docx in landscape orientation


In [19]:
import io
from google.cloud import vision
from docx import Document
from IPython.display import Markdown, display

def detect_text(path):
    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(image=image)
    texts = response.text_annotations
    ocr_text = []

    for text in texts:
        ocr_text.append(f"\r\n{text.description}")

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return ocr_text

def save_text_to_docx(text, output_filename):
    doc = Document()
    paragraphs = text.split('\n')
    for paragraph in paragraphs:
        doc.add_paragraph(paragraph.strip())
    doc.save(output_filename)

def display_markdown_with_breaks(text):
    display(Markdown(text))

# Main execution
image_path = "/content/a/Piccolomini_page_10.jpg"
initial_name = "Piccolomini_page_10"  # You can change this as needed

# Detect text
ocr_result = detect_text(image_path)

# Save to docx
output_filename = f"{initial_name}_ocr.docx"
save_text_to_docx(ocr_result[0], output_filename)

print(f"OCR text has been saved to {output_filename}")

# Display the text in Markdown format (optional)
display_markdown_with_breaks(ocr_result[0])

OCR text has been saved to Piccolomini_page_10_ocr.docx



ME CHA N.
II
re ipfa ab omni materia feiunctas ac feparatas confyde/
rat, mentes fcilicet illas fimplices, nulliq mutabilitati
obnoxias: veluti intelligentia funt, & in primis Deus ip
fe optimus maximus. reliqua vero Metaphifica pars in
comunifsimis verfatur: veluti in eo quod eft, cuius par/
tes atq proprietates perfcrutatur. Quo fit, vt ex hac
Metaphifica parte, omnia qua tanquam cognita in ca
teris fcientiis affummuntur, fi á quopiam abnegentur,
probari & oftendi pofsint. quamobrem circa eadem la/
borant Metaphificus & Dialecticus, circa communifsi/
ma fcilicet: diuerfa tamen ratione, vt apud Alexandru
fiue Ephefium Michaelem, abunde videri poteft. Sed
de Diuina,& naturali philofophia hactenus,cum ad rem
modo noftram non pertineant. Cæterum Mathemati
ca, quæ contemplatricis philofophiæ tertia reftat pars,
quia quantum, magnitudinem fcilicet & multitudinem,
eafqz nullo habito refpectu ad materiam in quá firmen/
tur, confyderat: iccirco duæ funt eius partes, altera nu
merum contemplans, Arithmetica fcilicet: reliqua ve/
ro continuam magnitudinem fpeculans, quam Geome
triam nuncupamus,quę tametfi numerus ex continui di/
uifione nafcatur, pofterior tamen atqz ignobilior eft, q
fit arithmetica, multas ob caufas, quac apud Boetium in
Arithmeticis haberi poffunt. Vtraq; autem harum ma
thematica partium, alias rurfus complectitur partes: no
quidem fellularias artes ( vt quidam volunt, qui illas
fub decimo Euclidis libro non recte collocant, in quo
libro potentia tantum habetur magnitudinum) fed
B ii Arith/